## AMLD Workshop Notebook 3

In this notebook, you will combine what you learned in notebooks 1 and 2 to train a federated machine learning task.

## 🎯 OBJECTIVE

Train the same machine learning model collaboratively across multiple institutions using federated learning, without transferring raw patient data outside local environments.

<div style="background-color: rgba(182, 255, 18, 0.15); border-left: 5px solid #B6FF12; padding: 15px; margin: 10px 0;">
<h3> Real-world context</h3>
<p>In healthcare, data is often legally and ethically constrained to remain within institutional or national boundaries. As a result, pooling datasets into a single central location for AI training is frequently infeasible. Yet, developing robust and generalizable models requires access to diverse populations and larger sample sizes than any single institution can typically provide.

Federated learning addresses this tension by allowing each institution to train a model locally and share only model updates rather than raw data. In practice, this requires more than just algorithms: it depends on secure execution environments, identity management, authorization workflows, and auditability across all participating sites.

In this setup, CHORUS provides the secure processing environment and governance controls at each hospital node, while Tune Insight coordinates the federated training process across institutions. Together, they make cross-institution AI development operationally feasible while respecting data sovereignty and regulatory constraints.
</p>
</div>

## ⚙️ IMPLEMENTATION

### Setup

In [ ]:
import pandas as pd
import uuid

from tuneinsight import Diapason, models
from tuneinsight.computations import HybridFL
import tuneinsight.utils.time_tools as time

#### Create clients

In [ ]:
from amld_setup import *

# TODO: Please update the credentials below with those provided to you.
%env TI_USERNAME=amld-workshopX@tuneinsight.com
%env TI_PASSWORD=AMLD_workshop_X

In [ ]:
client = Diapason.from_env()

In [ ]:
client.healthcheck()

#### Create and share the project

Projects are the main unit of collaboration in Tune Insight projects. In a project, you will define the computation to run in a federated setting, and set the datasource used by your instance. Other participants will also choose the data used by their instance. Once everything is set up, the federated analysis can be run using data from all instances, without centralizing the data.

In [ ]:
PROJECT_NAME = f"project-3-{uuid.uuid4()}"

project = client.new_project(name=PROJECT_NAME, clear_if_exists=True)
project.share()

## Load the dataset

In [ ]:
data_path = "data/data_0.csv"

In [ ]:
df = pd.read_csv(data_path)
df

In [ ]:
# Feel free to play around with the data if you want.

### Split the dataset into training and validation sets

In [ ]:
df["split"] = "train"
df.loc[df.sample(frac=0.2, random_state=42).index, "split"] = "val"
df

Upload the data to the instance and set it on the project.

In [ ]:
datasource = client.new_datasource(dataframe=df, name=f"patient_data_{uuid.uuid4()}", clear_if_exists=True)

In [ ]:
project.set_datasource(datasource)

### Task Definition

In this notebook, we will define a machine learning task with the `ti-models` library, similar to what you did in notebook 2.

In [ ]:
from torch import nn
from ti_models.models.ti_model import TIModel
from ti_models.preprocessing.preprocessing import Preprocessing, InputType
from ti_models.preprocessing.datasets.csv_dataset import CSVDataset
from ti_models.trainer.ti_trainer import TITrainer
from ti_models.trainer.ti_loss import TILoss
from ti_models.trainer.ti_optimizer import TIOptimizer, OptimizerType

Build the model architecture in PyTorch

In [ ]:
INPUT_DIM = 4
N_CLASSES = 2

torch_model = nn.Sequential(
    nn.BatchNorm1d(INPUT_DIM, affine=False, track_running_stats=True),
    nn.Linear(INPUT_DIM, N_CLASSES),
    nn.Sigmoid(),
)

Construct the `TIModel` wrapper that adds constraints to data inputs

In [ ]:
preprocessing = Preprocessing(input_type=InputType.TABULAR, input_shape=(INPUT_DIM,))

model = TIModel(
    name="LogisticRegression",
    description="Logistic Regression classifier for prostate cancer detection task",
    n_classes=N_CLASSES,
    torch_model=torch_model,
    preprocessing=preprocessing,
)

Build the federated trainer `TITrainer` that specifies loss, optimizer and batch size

In [ ]:
LEARNING_RATE = 0.1
MOMENTUM = 0.9
BATCH_SIZE = 16

loss = TILoss(loss_func=nn.CrossEntropyLoss())

optimizer = TIOptimizer(
    optimizer_type=OptimizerType.SECURE_SGD,
    lr=LEARNING_RATE,
    momentum=MOMENTUM
)

trainer = TITrainer(
    model=model,
    loss=loss,
    optimizer=optimizer,
    batch_size=BATCH_SIZE,
)

In [ ]:
print(trainer)

#### Start with a preliminary local training on your dataset

Create the datasets for local training

In [ ]:
train_df = df[df["split"] == "train"].drop(columns=["split"])
val_df = df[df["split"] == "val"].drop(columns=["split"])

train_dataset = CSVDataset(df = train_df)
val_dataset = CSVDataset(df = val_df)

Create a copy of the trainer to avoid modifications on the federated trainer

In [ ]:
local_trainer = TITrainer.unmarshal_binary(trainer.marshal_binary())

Train the model for 3 epochs using the trainer

In [ ]:
local_trainer.train(
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    epochs=3,
    eval_after_epoch=True,
    logging_frequency=10,
)

In [ ]:
local_accuracy = round(local_trainer.history.get_curve("accuracy")[-1][2], 4)
print("Local accuracy:", local_accuracy)

### Setup the parameters for Federated Learning

We do 3 collective rounds of 1 local epoch each

In [ ]:
params = models.HybridFLGenericParams(
    fl_rounds=3,
    num_workers=2,
    strategy = models.aggregation_strategy.AggregationStrategy.CONSTANT
)

ml_params = models.HybridFLMachineLearningParams(
    local_epochs=1,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    momentum=MOMENTUM
)

Define the computation (Federated Learning) on the project.

In [ ]:
hybrid_fl = HybridFL(
    project=project,
    task_id = "logreg",
    trainer=trainer,
    params=params,
    spec_params= ml_params,
)
hybrid_fl.max_timeout = 300 * 60 * time.SECOND

Clients authorize the project

In [ ]:
project.request_authorization()

Here you can get a quick summary of the project:

In [ ]:
project.display_overview()

In [ ]:
project.display_datasources()

## Run the training

This will run a federated learning on the network of four instances (three contributing, and a coordinating root node).

Note: you might experience the following error

> `InternalError: error happened internally: unexpected error: please contact the instance's administrator`

if that is the case, please call one of the organizers for assistance.

In [ ]:
hybrid_fl.run()

### Retrieve the result history

In [ ]:
results = project.fetch_results()[-1][1]

In [ ]:
hybrid_fl.display_results(results.history)

In [ ]:
import json

federated_accuracy = round(json.loads(results.history.metrics[-1])["accuracy"][-1][1], 4)
print("Local accuracy:", local_accuracy)
print("Federated accuracy:", federated_accuracy)